<a href="https://colab.research.google.com/github/valterlucena/recuperacao-informacao/blob/master/index-query/index_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
nltk.download('rslp')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import collections
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
DATA_URL = 'https://raw.githubusercontent.com/valterlucena/ri_lab_01/master/output/results.csv'
news = pd.read_csv(DATA_URL).replace(np.nan, '', regex=True)

In [0]:
toker = RegexpTokenizer('''\w+[-']*\w*''')
stop_words = stopwords.words('portuguese')

def isValid(token):
  return token not in stop_words and not bool(re.search(r'\d', token))

def build_index(documents):
  index = {}
  n = 0
  for document in documents:
    n += 1
    tokens = [token for token in toker.tokenize(document.lower()) if isValid(token)]
    for token in tokens:
      occurrence = tokens.count(token)
      if token not in index:
        index[token] = {}
      if n not in index[token]:
        index[token][n] = occurrence
  return index

index = build_index(news.text)

data = {'token':list(index.keys()), 'oc_list':list(index.values())}
df = pd.DataFrame(data)
#df.to_csv('index.csv')

In [57]:
def document_at_a_time_retrieval(Q, I, k):
  L = []
  R = []
  for term in Q.split():
    li = I[term]
    L.append(li)
  documents = []
  for li in L:
    documents.extend([document for document in li.keys() if document not in documents])
  for document in documents:
    sd = 0
    for li in L:
      for d,f in li.items():
        if document == d:
          sd += f
    R.append((document, sd))
  R = sorted(R, key = lambda x: x[1], reverse=True)[0:k]
  return R

document_at_a_time_retrieval('presidente lula', index, 20)


[(117, 78),
 (120, 22),
 (29, 19),
 (99, 17),
 (64, 16),
 (50, 11),
 (75, 9),
 (111, 9),
 (123, 9),
 (65, 8),
 (45, 7),
 (98, 6),
 (136, 6),
 (15, 5),
 (89, 5),
 (94, 5),
 (125, 5),
 (10, 4),
 (93, 4),
 (37, 3)]